<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/BSE_large_mid_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install quandl

In [ ]:
import quandl
import pandas as pd
import numpy as np
import requests
from quandl.errors.quandl_error import NotFoundError
import matplotlib.pyplot as plt

In [ ]:
#@title Download from BSE
def download_from_BSE(datadump,startdate):
    data= {}
    df=pd.DataFrame()
    
    print(f'len of datadump {len(datadump)}')
    try:
      for i in range(len(datadump)):
          data[datadump[i]]= quandl.get('BSE/BOM'+datadump[i],start_date=startdate, authtoken="1Mz3xLNFNdUpKENCkArB")['Close']
          
          
      i=0
      for key , value in data.items():
        df.insert(i, key, value)
        i=i+1
    except NotFoundError:
      print('Not Found')

    return df

In [ ]:
#@title Rejected Scrips
rejectlst= ['543253','543213','543210','543233','543252','543249','543212','543242','543248','543066','543232','542759','543223','543238',\
            '543243','543235','543245','543220','543237','543240','543227','543228','543066',\
            ]

In [ ]:
#@title Sectors
def get_bse_sector_list(data):

  
  if data=='LargeCap':
    url ='https://www.bseindia.com/sensex/Index_Contribution.aspx?index_code=93'
  if data== 'MidCap':
    url = 'https://www.bseindia.com/sensex/Index_Contribution.aspx?index_code=81' 
  if data== 'SmallCap':
    url = 'https://www.bseindia.com/sensex/Index_Contribution.aspx?index_code=82'
  if data == 'BSE100':
    url = 'https://www.bseindia.com/sensex/Index_Contribution.aspx?index_code=22'
  
  header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
  }

  r = requests.get(url, headers=header)
  data_dict={}
  dfs = pd.read_html(r.text)  
  dataframe=dfs[1]

  
  #scrip_code = [int(i) for i in scrip_code]
  return dataframe

In [ ]:
dataframe=get_bse_sector_list('MidCap')

In [ ]:
dataframe.columns=dataframe.iloc[0]

In [ ]:
dataframe=dataframe[1:]

In [ ]:
dataframe.to_csv('MidCap.csv')

In [ ]:
#@title Feature Engineering
def do_feature_engineering(datadump,startdate,rejectlst=rejectlst):
  datadump=[i for i in datadump if i not in rejectlst]
  data=download_from_BSE(datadump,startdate)
  data_copy=data.copy()
  print(data.shape)
  for i in range(len(data.iloc[0])):
    
    varname = 'ema_'+str(i)
    
    data[varname]=data[data.columns[i]]-data[data.columns[i]].rolling(window=50).mean()
    
  new_df = data.loc[:, ~data.columns.isin(data_copy.columns.tolist())]
    
  new_df=new_df.mask(new_df>0,1)
  new_df=new_df.mask(new_df<=0,0)
  above_50_dma=new_df.sum(axis=1) 
  Ad_dataframe=pd.DataFrame(above_50_dma) 
  Ad_dataframe.columns=['sum']
  Ad_dataframe['macd']=Ad_dataframe['sum'].ewm(span=20, adjust=False).mean()-Ad_dataframe['sum'].ewm(span=50, adjust=False).mean()
  Ad_dataframe['signal']=Ad_dataframe['macd'].ewm(span=15,adjust=False).mean()
  Ad_dataframe['hist']=Ad_dataframe['macd']-Ad_dataframe['signal']
    
  return Ad_dataframe

In [ ]:
#@title Feature Engineering 52 wk
def do_feature_engineering_52wk(datadump,startdate,rejectlst=rejectlst):
  datadump=[i for i in datadump if i not in rejectlst]
  data=download_from_BSE(datadump,startdate)
  data_copy=data.copy()
  print(data.shape)
 

  lst=[]
  for i in range(len(data.iloc[0])):
    
    #varname = 'ema_'+str(i)
    
    #data[varname]=data[data.columns[i]]-data[data.columns[i]].rolling(window=50).mean()
    
    varn='52wkhigh'+str(i)
    data[varn]=data[data.columns[i]].rolling(252).max()
    var='var'+str(i)
    data[var]=np.where(data[data.columns[i]]>=data[varn],1,0)
    var='var'+str(i)
    lst.append(var)
      
  new_df = data.loc[:,data.columns.isin(lst)]
    
  new_df=new_df.mask(new_df>0,1)
  new_df=new_df.mask(new_df<=0,0)
  above_50_dma=new_df.sum(axis=1) 
  Ad_dataframe=pd.DataFrame(above_50_dma) 
  Ad_dataframe.columns=['sum']
  Ad_dataframe['macd']=Ad_dataframe['sum'].ewm(span=20, adjust=False).mean()-Ad_dataframe['sum'].ewm(span=50, adjust=False).mean()
  Ad_dataframe['signal']=Ad_dataframe['macd'].ewm(span=15,adjust=False).mean()
  Ad_dataframe['hist']=Ad_dataframe['macd']-Ad_dataframe['signal']
  return Ad_dataframe

In [ ]:
#@title Draw Plot
def draw_plot(labels,df,nifty,buy,sell):
  fig, ax = plt.subplots(3,1, figsize=(15,9),sharex=True)
  ax[0].plot(df.loc[:, :].index, df.loc[:, 'sum'], label=labels)
  ax[2].plot(nifty.loc[:, :].index, nifty.loc[:, 'Close'], label='Nifty')
  ax[2].scatter(nifty.iloc[buy].index,nifty.iloc[buy].Close,marker='^',label='Buy',color='g')
  ax[2].scatter(nifty.iloc[sell].index,nifty.iloc[sell].Close,marker='v',label='Sell',color='r')
  ax[2].plot(nifty['Close'].ewm(span=50,adjust=False).mean())
  ax[0].plot(df['sum'].ewm(span=50, adjust=False).mean(),linewidth=2, markersize=4 ,label='50EMA' ,color='r')
  ax[0].plot(df['sum'].ewm(span=20, adjust=False).mean(),linewidth=1, markersize=4 ,label='20EMA' ,color='g')
  ax[1].plot(df['macd'],label='MACD')
  ax[1].plot(df['signal'],label='Signal')
  #ax[1].hist(df['hist'],label='Histogram')
  ax[0].legend(loc='upper left')
  ax[0].set_ylabel(labels)
  ax[1].legend('MACD')
  ax[1].legend(loc='upper left')
  ax[2].legend('Nifty')
  #ax[0].axhline(y=50 , color ='r')
  #ax[0].axhline(y=80 , color ='c')
  #ax[0].axhline(y=40 , color ='b')
  #ax[0].axhline(y=20 , color ='g')
  ax[1].axhline(y=0 , color='r')
  fig.suptitle(labels)
  plt.show()

In [ ]:
#@title plot buy sell signal
def plot_buy_sell_signal(df):
  buy=[]
  sell=[]
  for i in range(len(df)):
    if df.macd.iloc[i]>df.signal.iloc[i] and df.macd.iloc[i-1]<df.signal.iloc[i-1] :
      buy.append(i)
    elif df.macd.iloc[i]<df.signal.iloc[i] and df.macd.iloc[i-1]>df.signal.iloc[i-1]:
      sell.append(i)
  
  return buy ,sell

In [ ]:
startdate='2019-01-01'


# Fill The Column Below

In [ ]:
#@title Get All Data
def get_all_in_one(data,startdate,rejectlst=rejectlst):
  datadump=get_bse_sector_list(data) ##### Only Fill This part ##########
  nifty=quandl.get('BSE/'+ 'BSE100',start_date=startdate, authtoken="1Mz3xLNFNdUpKENCkArB")
  df_fin_eng=do_feature_engineering(datadump,startdate,rejectlst=rejectlst)
  buy,sell=plot_buy_sell_signal(df_fin_eng)
  d=nifty.reset_index()
  b=buy[-1:] 
  s=sell[-1:]
  print(f' ----------------')
    
  if b>s:
    print(f'|Buy : {data}   |')
    print(f'|{str(d.iloc[b].Date.values[0])[:10]}     |')
    print(f'|{d.iloc[b].Close.values.tolist()[0]}       |')
  if s>b:
    print(f'|Sell : {data}    |')
    print(f'|{str(d.iloc[s].Date.values[0])[:10]}     |')
    print(f'|{d.iloc[s].Close.values.tolist()[0]}       |')
  print(f' ----------------')  
  lbls= 'Above 50 EMA : '+str(data)
  draw_plot(lbls,df_fin_eng,nifty,buy,sell)


In [ ]:
lst=['LargeCap','MidCap','SmallCap','BSE100']


In [ ]:
for data in lst:
  get_all_in_one(data,startdate,rejectlst=rejectlst)
  

len of datadump 80


KeyboardInterrupt: ignored

In [ ]:
nifty=quandl.get('BSE/'+ 'BSE100',start_date=startdate, authtoken="1Mz3xLNFNdUpKENCkArB")

In [ ]:
nifty.tail(2)